1. load the function
2. read the data (vcfs, parsed.flow, parsed.geno, args)
3. make plots
4. save

# Load functions

In [1]:
suppressMessages(library('vcfR'))
suppressMessages(library('data.table'))
suppressMessages(library('RColorBrewer'))
suppressMessages(library('kinship2'))
suppressMessages(library('plotly'))
suppressMessages(library('htmltools'))
suppressMessages(library('GenomicRanges'))
suppressMessages(library('DNAcopy'))
suppressMessages(library('knitr'))
suppressMessages(library('stringr'))
suppressMessages(library(htmlwidgets))

In [5]:
args <- list(
  ## mandatory arguments
  vcf.file=c(),
  sample.ids=c(),
  
  ## important optional arguments
  father.ids=c(),
  mother.ids=c(),
  genders=c(),
  run.merlin=T,
  cytoband.file=c(),
  
  ## variant inclusion arguments: filter 1
  dp.hard.limit.ids=c(),
  dp.hard.limit=10,
  af.hard.limit.ids=c(),
  af.hard.limit=0,
  dp.soft.limit.ids=c(),
  dp.soft.limit=10,
  
  ## variant inclusion arguments: filter 2
  keep.informative.ids=c(),
  keep.hetero.ids=c(),
  
  ## sample/disease annotation
  regions=c(),
  reference.ids=c(),
  carrier.ids=c(),
  affected.ids=c(),
  nonaffected.ids=c(),
  info=c(),
  
  ## BAF profiles
  baf.ids=c(),
  
  ## merlin profiles
  merlin.model='best',
  min.seg.var=5,
  min.seg.var.X=15,
  window.size.voting=10000000,
  window.size.voting.X=c(),
  keep.chromosomes.only=T,
  keep.regions.only=F,
  concordance.table=T,
  
  ## remaining features
  out.dir=c('./'),
  fam.id='hopla',
  X.cutoff=1.5,
  Y.cutoff=.6,
  window.size=1000000,
  regions.flanking.size=2000000,
  limit.baf.to.P=F,
  limit.pm.to.P=F,
  value.of.P=.25,
  color.palette='Paired',
  dot.factor=2,
  self.contained=F,
  cairo=F
)

In [6]:
# functions for loading arguments

trim <- function (x) gsub("^\\s+|\\s+$", "", x)

format.value <- function(arg, value){
  if (!(arg %in% names(args))){
    cat(paste0('ERROR: Argument --', arg, ' does not exist.\n'))
    quit(status=0)
  }
  
  numeric.args <- c('dp.hard.limit', 'af.hard.limit', 'dp.soft.limit',
                    'min.seg.var', 'min.seg.var.X', 'window.size.voting',
                    'window.size.voting.X', 'X.cutoff', 'Y.cutoff', 'window.size',
                    'regions.flanking.size', 'value.of.P', 'dot.factor')
  boolean.args <- c('run.merlin', 'keep.chromosomes.only', 'keep.regions.only',
                    'concordance.table', 'limit.baf.to.P', 'limit.pm.to.P',
                    'self.contained', 'cairo')
  
  value = sapply(strsplit(value, ',')[[1]], function(x) trim(x))
  
  ## replace NAs
  value[value %in% c('', 'NA')] = NA
  
  ## tranform when needed
  if (arg %in% numeric.args) value = as.numeric(value)
  if (arg %in% boolean.args) value = as.logical(value)
  return(value)
}

get.file.args <- function(settings.file){
  if (!file.exists(settings.file)){
    cat('ERROR: File given by --settings does not exist. Please Correct.\n')
    quit(status=0)
  }
  at.info = F
  for (line in suppressWarnings(readLines(settings.file))){
    ## info parsing
    if (!at.info) line = gsub("'", '', gsub('"', '', line))
    if (at.info) line = gsub('\t', '    ', line)
    if (line == 'end.info'){ at.info = F ; next }
    if (at.info){ args$info <- c(args$info, line) ; next }
    if (line == 'start.info'){ at.info = T ; next }
    
    ## skip comments
    if (substr(trim(line), 1, 1) %in% c('#', '')) next
    line <- trim(strsplit(line, '#', fixed = T)[[1]][1])
    
    ## parse argument
    arg <- trim(strsplit(line, '=')[[1]][1])
    value <- strsplit(line, '=')[[1]][2]
    value <- format.value(arg, value)
    
    if (substr(line, nchar(line), nchar(line)) != '=' & 
        grepl('=', line, fixed = T)) args[[arg]] <- value
  }
  return(args)
}

post.process.args <- function(args){
  no.u.mask <- !sapply(args$sample.ids, function(x) toupper(substr(x,1,1)) == 'U' &
                         !is.na(suppressWarnings(as.numeric(substr(x,2,999)))))
  
  if (!length(args$genders)) args$genders = rep(NA, length(args$sample.ids))
  if (!length(args$mother.ids)) args$mother.ids = rep(NA, length(args$sample.ids))
  if (!length(args$father.ids)) args$father.ids = rep(NA, length(args$sample.ids))
  
  args$samples.no.u <- args$sample.ids[no.u.mask]
  args$samples.u <- args$sample.ids[!no.u.mask]
  
  not.last.in.line <- unique(c(args$mother.ids, args$father.ids))
  not.last.in.line <- not.last.in.line[not.last.in.line %in% args$samples.no.u]
  last.in.line <- args$samples.no.u[!args$samples.no.u %in% not.last.in.line]
  
  if (length(args$samples.no.u) == 1){
    not.last.in.line = args$samples.no.u
    last.in.line = args$samples.no.u
  }
  
  if (!length(args$dp.hard.limit.ids)) args$dp.hard.limit.ids = not.last.in.line
  if (!length(args$af.hard.limit.ids)) args$af.hard.limit.ids = not.last.in.line
  if (!length(args$dp.soft.limit.ids)) args$dp.soft.limit.ids = last.in.line
  if (!length(args$window.size.voting.X)) args$window.size.voting.X = args$window.size.voting
  if (!length(args$min.seg.var.X)) args$min.seg.var.X = args$min.seg.var
  
  man.args <- c('vcf.file', 'sample.ids')
  for (arg in names(args)){
    if (!length(args[[arg]]) & arg %in% man.args){
      cat(paste0('ERROR: Argument --', arg, ' is mandatory. Please provide.\n'))
      quit(status=0)
    }
  }
  
  not.in.error <- function(arg){
    not.in <- function(xs, ys){
      for (x in xs){
        if (!(x %in% ys) & !is.na(x)){
          return(x)
        }
      }
      return(NULL)
    }
    if (length(not.in(args[[arg]], args$sample.ids))){
      cat(paste0('ERROR: Fetched from argument --', arg,', \'', not.in(args[[arg]], args$sample.ids),
                 '\' could not be found in the provided --sample.ids. Please correct.\n'))
      quit(status=0)
    }
  }
  
  cat('Selected parameters ...\n')
  for (arg in names(args)[!(names(args) %in% c('samples.u', 'samples.no.u'))]){
    if (!length(args[[arg]])) next
    cat(paste0('  ... ', arg, ': ', paste(args[[arg]], collapse = ','), '\n'))
    
    if (any(is.na(args[[arg]])) & !(arg %in% c('father.ids', 'mother.ids', 'genders'))){
      cat(paste0('ERROR: No \'NA\' allowed in argument --', arg,'. Please correct.\n'))
      quit(status=0)
    }
    
    if (arg == 'sample.ids'){
      if (length(args$sample.ids) > 1){
        if (!length(which(!is.na(args$mother.ids))) & !length(which(!is.na(args$father.ids)))){
          cat('ERROR: More than one sample is given in --sample.ids. Please provide their relation using argument(s)', 
              '--father.ids and/or --mother.ids. Otherwise, run separately.\n')
          quit(status=0)
        }
      }
    }
    
    if (arg == 'genders'){
      if (!all(args$genders %in% c('M', 'F', NA))){
        cat('ERROR: Argument --genders should be coded as \'M\', \'F\' or \'NA\'. Please correct.\n')
        quit(status=0)
      }
    }
    
    for (same.length.arg in c('father.ids', 'mother.ids', 'genders')){
      if (arg == same.length.arg){
        if (!(length(args$sample.ids) == length(args[[same.length.arg]]))){
          cat(paste0('ERROR: Arguments --sample.ids and --', same.length.arg,' should be of the same length. Please correct.\n'))
          quit(status=0)
        }
      }
    }
    
    if (arg %in% c('father.ids', 'mother.ids', 'dp.hard.limit.ids', 'af.hard.limit.ids',
                  'dp.soft.limit.ids', 'keep.informative.ids', 'keep.hetero.ids',
                  'reference.ids', 'carrier.ids', 'affected.ids',
                  'nonaffected.ids', 'baf.ids')){
      not.in.error(arg)
    }
    
    if (arg %in% c('dp.hard.limit.ids', 'af.hard.limit.ids', 'dp.soft.limit.ids',
                   'keep.informative.ids', 'keep.hetero.ids', 'baf.ids')){
      if (length(intersect(args[[arg]], args$samples.u))){
        cat(paste0('ERROR: \'U\' IDs not allowed in --', arg, '. Please correct.\n'))
        quit(status=0)
      }
    }
    
    if (arg == 'run.merlin' & args$run.merlin){
      if (length(args$sample.ids) == 1){
        cat(paste0('WARNING: Only one sample provided. Setting --run.merlin FALSE.\n'))
        args$run.merlin = F
      }
      if ((Sys.which('merlin') == '' | Sys.which('minx') == '') & args$run.merlin){
        cat(paste0('WARNING: Merlin executables folder could not be located in $PATH. Setting --run.merlin FALSE.\n'))
        args$run.merlin = F
      }
    }
    
    if (arg == 'keep.informative.ids'){
      if (!(length(args[[arg]]) %in% c(0,2))){
        cat(paste0('ERROR: No or two samples should be given at --keep.informative.ids. Please correct.\n'))
        quit(status=0)
      }
    }
    
    if (arg == 'merlin.model'){
      if (!(args$merlin.model %in% c('sample', 'best'))){
        cat('ERROR: Argument --merlin.model should be coded as \'sample\' or \'best\'. Please correct.\n')
        quit(status=0)
      }
    }
    
    if (arg == 'vcf.file'){
      if (!file.exists(args$vcf.file)){
        cat('ERROR: the file given by --vcf.file does not exist. Please correct.\n')
        quit(status=0)
      }
    }
    if (arg == 'cytoband.file'){
      if (length(args$cytoband.file) & !file.exists(args$cytoband.file)){
        cat('ERROR: the file given by --vcf.file does not exist. Please correct.\n')
        quit(status=0)
      }
    }
    if (arg == 'value.of.P'){
      if (args$value.of.P <= 0 | args$value.of.P > 1){
        cat('ERROR: --value.of.P should be within ]0, 1]. Please correct.\n')
        quit(status=0)
      }
    }
    if (arg == 'af.hard.limit'){
      if (args$af.hard.limit < 0 | args$af.hard.limit >= 1){
        cat('ERROR: --af.hard.limit should be within [0, 1[. Please correct.\n')
        quit(status=0)
      }
    }
  }
  
  add.annot <- function(letter, annot = NULL){
    if (!is.null(annot)){
      args$samples.out[args$sample.ids %in% args[[annot]]] <- paste0(args$samples.out[args$sample.ids %in% args[[annot]]],
                                                                     ' (', letter, ')')
    } else {
      known.set <- unique(c(args$reference.ids, args$carrier.ids, args$affected.ids, args$nonaffected.ids))
      args$samples.out[!(args$sample.ids %in% known.set)] <- paste0(args$samples.out[!(args$sample.ids %in% known.set)],
                                                                    ' (', letter, ')')
    }
    return(args)
  }
  args$samples.out <- args$sample.ids
  args <- add.annot('R', 'reference.ids')
  args <- add.annot('C', 'carrier.ids')
  args <- add.annot('A', 'affected.ids')
  args <- add.annot('N', 'nonaffected.ids')
  
  return(args)
}

correct.profiles <- function(args, parsed.flow){
  is.corrected <- list()
  for (chr in chrs){
    is.corrected[[chr]] <- matrix(nrow = nrow(map.list[[chr]]), ncol = length(args$samples.no.u) * 2)
    for (i in 1:length(args$samples.no.u)){
      is.corrected[[chr]][,(i*2)-1] <- F
      is.corrected[[chr]][,i*2] <- F
    }
  }
  
  correct.vector.1 <- function(v, pos, max.distance){
    letters = unique(v)
    neighbours.i <- lapply(pos, function(p) c(which(abs(pos - p) <= max.distance)))
    neighbours.letters <- lapply(1:length(v), function(i) v[neighbours.i[[i]]])
    neighbours.pos <- lapply(1:length(v), function(i) abs(pos - pos[i])[neighbours.i[[i]]])
    neighbours.weights <- lapply(neighbours.pos, function(x) (max.distance * 2) / (x + max.distance) - 1)
    if (max.distance == 0) neighbours.weights <- lapply(1:length(v), function(x) 1)
    c.v <- sapply(1:length(v), function(i)
      letters[which.max(c(sum(neighbours.weights[[i]][neighbours.letters[[i]] == letters[1]]),
                          sum(neighbours.weights[[i]][neighbours.letters[[i]] == letters[2]], na.rm = T)))])
    return(c.v)
  }
  
  correct.vector.2 <- function(flow, geno, min.seg.var){
    breakpoints <- which(c('ZZ', flow) != c(flow, 'ZZ'))
    for (i in 1:(length(breakpoints)-1)){
      sequence <- breakpoints[i]:c(breakpoints[i+1]-1)
      if (length(which(geno[sequence] != 'NA')) > min.seg.var) next
      if (breakpoints[i]-1 != 0){
        letter = flow[breakpoints[i]-1] # previous segment when possible
      } else {
        letter = flow[breakpoints[i+1]] # next segment otherwise
      }
      flow[sequence] <- rep(letter, length(sequence))
    }
    return(flow)
  }
  
  if (args$window.size.voting != 0 | args$min.seg.var != 0){
    cat('Correcting haplotypes, working ...\n')
    for (chr in chrs[1:22]){
      cat(paste0('  ... at ', chr, '\n'))
      pos = map.list[[chr]][,2]
      for (i in 1:length(args$samples.no.u)){
        v = parsed.flow[[chr]][,i]
        a <- sapply(strsplit(v, '|', fixed = T), function(x) x[1])
        b <- sapply(strsplit(v, '|', fixed = T), function(x) x[2])
        c.a = a
        c.b = b
        if (args$min.seg.var != 0){
          c.a <- correct.vector.2(c.a, sapply(strsplit(parsed.geno[[chr]][,i], '|', fixed = T), function(x) x[1]),
                                  args$min.seg.var)
          c.b <- correct.vector.2(c.b, sapply(strsplit(parsed.geno[[chr]][,i], '|', fixed = T), function(x) x[2]),
                                  args$min.seg.var)
        }
        if (args$window.size.voting != 0){
          c.a <- correct.vector.1(c.a, pos, args$window.size.voting / 2)
          c.b <- correct.vector.1(c.b, pos, args$window.size.voting / 2)
        }
        parsed.flow[[chr]][,i] <- paste0(c.a, '|', c.b)
        is.corrected[[chr]][,(i*2)-1] <- a != c.a
        is.corrected[[chr]][,i*2] <- b != c.b
      }
    }
  }
  
  if (args$window.size.voting.X != 0 | args$min.seg.var.X != 0){
    if (args$window.size.voting == 0 & args$min.seg.var == 0) cat('Correcting haplotypes, working ...\n')
    chr = chrs[23]
    cat(paste0('  ... at ', chr, '\n'))
    pos = map.list[[chr]][,2]
    for (i in 1:length(args$samples.no.u)){
      v = parsed.flow[[chr]][,i]
      a <- sapply(strsplit(v, '|', fixed = T), function(x) x[1])
      b <- sapply(strsplit(v, '|', fixed = T), function(x) x[2])
      c.a = a
      c.b = b
      if (args$min.seg.var.X != 0){
        c.a <- correct.vector.2(c.a, sapply(strsplit(parsed.geno[[chr]][,i], '|', fixed = T), function(x) x[1]),
                                args$min.seg.var.X)
        c.b <- correct.vector.2(c.b, sapply(strsplit(parsed.geno[[chr]][,i], '|', fixed = T), function(x) x[2]),
                                args$min.seg.var.X)
      }
      if (args$window.size.voting.X != 0){
        c.a <- correct.vector.1(c.a, pos, args$window.size.voting.X / 2)
        c.b <- correct.vector.1(c.b, pos, args$window.size.voting.X / 2)
      }
      parsed.flow[[chr]][,i] <- paste0(c.a, '|', c.b)
      is.corrected[[chr]][,(i*2)-1] <- a != c.a
      is.corrected[[chr]][,i*2] <- b != c.b
    }
  }
  return(list(parsed.flow = parsed.flow, is.corrected = is.corrected))
}

In [7]:
# functions for making plot

get.haplo.profiles <- function(){
  get.breaks <- function(f, pos){
    changes <- cumsum(rle(f)$lengths)
    colors <- letter.colors[rle(f)$values]
    breakpoints <- sapply(changes, function(x) pos[x] + (pos[x+1] - pos[x]) / 2)
    breakpoints <- breakpoints[-length(breakpoints)]
    return(list(breakpoints = breakpoints, colors = colors))
  }
  
  filter.region <- function(frame, chr, whole.chromosome = F){
    if (length(args$regions)){
      for (region in args$regions){
        r.split <- strsplit(region, ':')[[1]]
        if (chr == r.split[1]){
          if (whole.chromosome) return(frame)
          s <- as.numeric(strsplit(r.split[2],'-')[[1]][1])
          s <- s - args$regions.flanking.size
          e <- as.numeric(strsplit(r.split[2],'-')[[1]][2])
          e <- e + args$regions.flanking.size
          return(frame[frame$x >= s & frame$x <= e,])
        }
      }
    }
    return(frame[-(1:nrow(frame)),])
  }
  
  add.marker.and.trace <- function(fig, y, f = 'f1', mar = 'y-down-open', symbol.offset = .3){
    for (i in 1:(length(breaks[[s]][[f]])-1)){
      if (i != 1){
        fig <- add_markers(fig, x = breaks[[s]][[f]][i], y = rep(y + symbol.offset, 2),
                           name = NULL, hoverinfo = "none",
                           marker = list(symbol = mar, color = colors[length(letters) + 1],
                                         size = args$dot.factor * 8),
                           type='scatter', mode='marker', inherit = F)
      }
      fig <- add_trace(fig, x = c(breaks[[s]][[f]][i], breaks[[s]][[f]][i+1]), y = rep(y, 2),
                       name = NULL, hoverinfo = "none",
                       line = list(color = breaks[[s]][[paste0(f, 'cols')]][i], width = args$dot.factor * 2),
                       type='scatter', mode='lines', inherit = F)
    }
    return(fig)
  }
  
  chr.lengths <- sapply(chrs, function(x) max(vcfs.filtered2[[1]]$POS[vcfs.filtered2[[1]]$CHROM == x]))
  
  haplo.profiles <- list()
  for (c in chrs){
    haplo.frame <- matrix(nrow = 0, ncol = 9)
    annot.list <- list()
    breaks <- list()
    for (s in args$samples.no.u){
      x <- map.list[[c]]$pos
      f1 <- sapply(strsplit(parsed.flow[[c]][,which(args$samples.no.u == s)], '|', fixed = T), function(x) x[1])
      f2 <- sapply(strsplit(parsed.flow[[c]][,which(args$samples.no.u == s)], '|', fixed = T), function(x) x[2])
      g1 <- sapply(strsplit(parsed.geno[[c]][,which(args$samples.no.u == s)], '|', fixed = T), function(x) x[1])
      g2 <- sapply(strsplit(parsed.geno[[c]][,which(args$samples.no.u == s)], '|', fixed = T), function(x) x[2])
      c1 <- is.corrected[[c]][,which(args$samples.no.u == s) * 2 - 1]
      c2 <- is.corrected[[c]][,which(args$samples.no.u == s) * 2]
      breaks[[s]]$f1 <- c(x[1], get.breaks(f1, x)$breakpoints, x[length(x)])
      breaks[[s]]$f1cols <- get.breaks(f1, x)$colors
      breaks[[s]]$f2 <- c(x[1], get.breaks(f2, x)$breakpoints, x[length(x)])
      breaks[[s]]$f2cols <- get.breaks(f2, x)$colors
      symbol <- rep(1, length(c1) * 2)
      symbol[c(c1,c2)] <- 0
      symbol[c(g1,g2) == 'NA'] <- 101 ## will show nothing
      id1 <- paste0(c, ':', map.list[[c]]$pos.out, ' (', g1, ')')
      id2 <- paste0(c, ':', map.list[[c]]$pos.out, ' (', g2, ')')
      y1 <- length(args$samples.no.u) * 3 - which(args$samples.no.u == s) * 3 + 1
      y2 <- length(args$samples.no.u) * 3 - which(args$samples.no.u == s) * 3
      haplo.frame.sub <- data.frame(c(x, x), c(rep(y1, length(x)), rep(y2, length(x))),
                                    c(id1, id2), c(letter.colors[f1],  letter.colors[f2]), symbol,
                                    stringsAsFactors = F, c(rep(which(args$samples.no.u == s) * 2 - 1, length(x)),
                                                            rep(which(args$samples.no.u == s) * 2, length(x))))
      annot.list[[which(args$samples.no.u == s)]] <- list(x = 1, y = y1 + 1,
                                                          text = args$samples.out[args$sample.ids == s], showarrow = F)
      if (all(c(g1, g2) == 'NA')) next
      haplo.frame <- rbind(haplo.frame, haplo.frame.sub)
    }
    
    ## raw data points
    
    colnames(haplo.frame) <- c('x', 'y', 'id', 'col', 'symbol', 'name')
    haplo.frame.for.plotly <- haplo.frame
    if (args$keep.chromosomes.only) haplo.frame.for.plotly <- filter.region(haplo.frame, c, whole.chromosome = T)
    if (args$keep.regions.only) haplo.frame.for.plotly <- filter.region(haplo.frame, c)
    p <- plot_ly(haplo.frame.for.plotly, x =~x, y =~y, text =~id, name=~name, marker = list(color=~col,
                                                                                            symbol=~symbol,
                                                                                            size = args$dot.factor * 4,
                                                                                            line=list(color=~col)),
                 type = 'scattergl', mode = 'markers', hoverinfo = 'text', height = 900 * length(args$samples.no.u),
                 hoverlabel=list(bgcolor=~col))
    
    ## layout
    
    p <- p %>% layout(xaxis = list(title = list(text=c, standoff=10), showticklabels = F,
                                   zeroline = F, showgrid = F), showlegend = F,
                      yaxis = list(title = '',showticklabels = F, zeroline = F, showgrid = F,
                                   fixedrange = T, range = c(-1, length(args$samples.no.u) * 3 + 1)),
                      annotations = annot.list)
    
    ## add traces and recombination points
    
    for (s in args$samples.no.u){
      y2 <- length(args$samples.no.u) * 3 - which(args$samples.no.u == s) * 3
      p <- add.marker.and.trace(p, length(args$samples.no.u) * 3 - which(args$samples.no.u == s) * 3 + 1)
      p <- add.marker.and.trace(p, length(args$samples.no.u) * 3 - which(args$samples.no.u == s) * 3, 'f2', 'y-up-open', symbol.offset = -.3)
    }
    
    ## add regions
    
    if (length(args$regions)){
      for (region in args$regions){
        if (c == strsplit(region, ':')[[1]][1]){
          tmp <- 1 ; names(tmp) <- c
          p <- mark.region(p, tmp, c(-.5, length(args$samples.no.u) * 3 + 1), region, chr.lengths)
        }
      }
    }
    
    haplo.profiles[[c]] <- p
    break
  }
  return (haplo.profiles)
}

get.breaks <- function(f, pos){
    changes <- cumsum(rle(f)$lengths)
    colors <- letter.colors[rle(f)$values]
    breakpoints <- sapply(changes, function(x) pos[x] + (pos[x+1] - pos[x]) / 2)
    breakpoints <- breakpoints[-length(breakpoints)]
    return(list(breakpoints = breakpoints, colors = colors))
}

add.marker.and.trace <- function(fig, y, f = 'f1', mar = 'y-down-open', symbol.offset = .3){
for (i in 1:(length(breaks[[s]][[f]])-1)){
  if (i != 1){
    fig <- add_markers(fig, x = breaks[[s]][[f]][i], y = rep(y + symbol.offset, 2),
                       name = NULL, hoverinfo = "none",
                       marker = list(symbol = mar, color = colors[length(letters) + 1],
                                     size = args$dot.factor * 8),
                       type='scatter', mode='marker', inherit = F)
  }
  fig <- add_trace(fig, x = c(breaks[[s]][[f]][i], breaks[[s]][[f]][i+1]), y = rep(y, 2),
                   name = NULL, hoverinfo = "none",
                   line = list(color = breaks[[s]][[paste0(f, 'cols')]][i], width = args$dot.factor * 2),
                   type='scatter', mode='lines', inherit = F)
}
return(fig)
}

mark.region <- function(fig, chr.cs, ylim, region, chr.lengths, plot.flanks = T){
  c <- strsplit(region, ':')[[1]][1]
  s <- as.numeric(strsplit(strsplit(region, ':')[[1]][2], '-')[[1]][1]) ; s <- max(s, 1)
  flank.s <- s - args$regions.flanking.size ; flank.s <- max(flank.s, 1)
  e <- as.numeric(strsplit(strsplit(region, ':')[[1]][2], '-')[[1]][2]) ; e <- min(e, chr.lengths[c])
  flank.e <- e + args$regions.flanking.size ; flank.e <- min(flank.e, chr.lengths[c])
  
  region.start <- paste0(c, ':', scales::comma(s, accuracy = 1))
  region.end <- paste0(c, ':', scales::comma(e, accuracy = 1))
  flank.start <- paste0(c, ':', scales::comma(flank.s, accuracy = 1))
  flank.end <- paste0(c, ':', scales::comma(flank.e, accuracy = 1))
  
  add.trace <- function(fig, xs, text, size = 1/2){
    fig <- fig %>% add_trace(x = xs, y = ylim, name = 'region',
                             hoverinfo = "text", line = list(color = colors[length(letters) + 1], width = args$dot.factor * size),
                             text = text, type='scatter', marker = list(color = colors[length(letters) + 1], size = .1),
                             mode='lines+markers', hoverlabel=list(bgcolor=colors[length(letters) + 1]), inherit = F)
    return(fig)
  }
  
  fig <- add.trace(fig, c(chr.cs[c] + s, chr.cs[c] + s), paste0(region.start, ' (region start)'))
  fig <- add.trace(fig, c(chr.cs[c] + e, chr.cs[c] + e), paste0(region.end, ' (region end)'))
  
  if (plot.flanks){
    fig <- add.trace(fig, rep(chr.cs[c] + flank.s, 2), paste0(flank.start, ' (flank start)'), 1/4)
    fig <- add.trace(fig, rep(chr.cs[c] + flank.e, 2), paste0(flank.end, ' (flank end)'), 1/4)
  }
  return(fig)
}

add.cytoband <- function(fig, chr, y, line.width = 4){
  s <- c()
  names <- c()
  cols <- c()
  for (i in 1:length(cytobands[[chr]])){
    xs <- c(cytobands[[chr]][[i]]$start, cytobands[[chr]][[i]]$end)
    col <- c('lightgrey', 'darkgrey')[[i %% 2 + 1]]
    if (any(sapply(c('acen', 'gvar', 'stalk'), function(x) grepl(x, cytobands[[chr]][[i]]$name)))) col <- 'black'
    fig <- add_trace(fig, x = xs, y = rep(y, 2),
                     name = NULL, hoverinfo = "none",
                     line = list(color = col, width = args$dot.factor * line.width),
                     type='scatter', mode='lines', inherit = F)
    seq <- seq(xs[1], xs[2], 10000000)
    s <- c(s, seq)
    names <- c(names, paste0(cytobands[[chr]][[i]]$name, ' (',
                             scales::comma(seq, accuracy = 1), ')'))
    cols <- c(cols, rep(col, length(seq)))
  }
  
  fig <- add_markers(fig, x = s, y = rep(y, length(s)), text=names, name = NULL,
                     hoverinfo='text', marker = list(color = cols, size = args$dot.factor * .1),
                     type='scatter', mode='marker', inherit = F)
  return(fig)
}

In [3]:
get.empty <- function(){
  p <- plot_ly(x = 0, y = 0, name = NULL, type = 'scatter', mode = 'markers',
                marker = list(color = 'white', size = .01), hoverinfo='skip')
  p <- p %>% layout(xaxis = list(title = '', showticklabels = F, zeroline = F, showgrid = F, fixedrange = T),
                    yaxis = list(title = '', showticklabels = F, zeroline = F, showgrid = F, fixedrange = T),
                    showlegend = F)
  return(p)
}


do.subplot <- function(plot.list, ncol = 1, panning = .03, margin = .02, overridehovermode = NULL){
  nrow <- ceiling(length(plot.list) / ncol)
  
  plot.list.ap <- lapply(1:(ncol + 2), function(x) get.empty())
  for (i in 1:nrow){
    plot.list.ap <- c(plot.list.ap, lapply(1, function(x) get.empty()))
    s <- (1 + (i - 1) * ncol)
    e <- (ncol + (i - 1) * ncol)
    real.e <- length(plot.list)
    plot.list.ap <- c(plot.list.ap, plot.list[s:min(real.e, e)])
    plot.list.ap <- c(plot.list.ap, lapply(1, function(x) get.empty()))
  }
  plot.list.ap <- c(plot.list.ap, lapply((length(plot.list.ap)+1):((ncol+2)*(nrow+2)), function(x) get.empty()))
  
  heights <- c(panning, rep((1 - 2*panning) / nrow, nrow), panning)
  widths <- c(panning, rep((1 - 2*panning) / ncol, ncol), panning)
  this.subplot <- subplot(plot.list.ap, nrows=nrow+2, margin=margin, titleY=T, titleX=T,
                          heights=heights, widths = widths)
  if (!is.null(overridehovermode)) this.subplot$x$layout$hovermode <- overridehovermode
  return(this.subplot)
}

# Read the data

In [8]:
setting <- '/home/jovyan/work/hopla/data/setting_haplotyping_demo.txt'

args <- get.file.args(setting)
args <- post.process.args(args)

dir.create(args$out.dir, showWarnings = F, recursive = T)
args$data.dir <- dirname(args$vcf.file)

chrs <- paste0('chr', c(1:22, 'X'))

ERROR: Argument --visualization does not exist.
Selected parameters ...
  ... vcf.file: /home/jovyan/work/hopla/inkyun/data/Proband_16_09932-gatk-haplotype-joint-annotated.vcf.gz
  ... sample.ids: U8,D2301776,D2303583,K2300654,K2300655,K2300656,K2300657
  ... father.ids: NA,NA,U8,U8,U8,U8,U8
  ... mother.ids: NA,NA,D2301776,D2301776,D2301776,D2301776,D2301776
  ... genders: M,F,M,NA,NA,NA,NA
  ... run.merlin: TRUE
  ... cytoband.file: /home/jovyan/work/hopla/data/cytoband.hg38.txt
  ... dp.hard.limit.ids: D2301776,D2303583
  ... dp.hard.limit: 10
  ... af.hard.limit.ids: D2301776,D2303583
  ... af.hard.limit: 0.25
  ... dp.soft.limit.ids: K2300654,K2300655,K2300656,K2300657
  ... dp.soft.limit: 10
  ... regions: chrX:53176283-53225422
  ... carrier.ids: D2301776
  ... affected.ids: D2303583
  ... info: Disease:Intellectual developmental disorder KDM5C,Inheritance:XLR,Sequencing note:
  ... baf.ids: K2300654,K2300655,K2300656,K2300657
  ... merlin.model: best
  ... min.seg.var: 5
  ... 

In [9]:
# load vcfs

vcfs_merged <- read.csv(file.path(args$data.dir, 'vcfs.csv'))
vcfs_merged <- vcfs_merged[,-1]
vcfs <- list()

for (i in 1:length(vcfs_merged)) {
    column_name = colnames(vcfs_merged)[i]
    sample_name <- str_split(column_name, '\\.', n=2)[[1]]
    sampleID <- sample_name[1]
    col <- sample_name[2]
  
    if (is.null(vcfs[[sampleID]])) {
        vcfs[[sampleID]] <- data.frame(vcfs_merged[[column_name]])
    } else {
        vcfs[[sampleID]] <- cbind(vcfs[[sampleID]], vcfs_merged[[column_name]])
    }

    if (i%%12 == 0) {
        colnames(vcfs[[sampleID]])[12] <- sample_name[2]
    } else {
        colnames(vcfs[[sampleID]])[i%%12] <- sample_name[2]
    }
}

In [11]:
# extract vcfs.filtered and vcfs.filtered2 from vcfs

vcfs.filtered <- list()
vcfs.filtered2 <- list()

for (i in 1:length(vcfs)) {
    last.col.num <- length(vcfs[[1]])-2
    vcfs.filtered[[i]] <- vcfs[[i]][vcfs[[i]]$On_filter1 == 1, ][1:last.col.num]
    vcfs.filtered2[[i]] <- vcfs[[i]][vcfs[[i]]$On_filter2 == 1, ][1:last.col.num]
    vcfs[[i]] <- vcfs[[i]][1:last.col.num]
    colnames(vcfs[[i]])[ncol(vcfs[[i]])] <- "POS.OUT"

    hom.ref <- which(vcfs.filtered[[i]]$GT == '0/0')
    het.alt <- which(vcfs.filtered[[i]]$GT == '0/1')
    hom.alt <- which(vcfs.filtered[[i]]$GT == '1/1')
    
    vcfs.filtered[[i]]$GENO <- 'N/N'
    vcfs.filtered[[i]]$GENO[hom.ref] <- paste0(vcfs.filtered[[i]]$REF, '/', vcfs.filtered[[i]]$REF)[hom.ref]
    vcfs.filtered[[i]]$GENO[het.alt] <- paste0(vcfs.filtered[[i]]$REF, '/', vcfs.filtered[[i]]$ALT)[het.alt]
    vcfs.filtered[[i]]$GENO[hom.alt] <- paste0(vcfs.filtered[[i]]$ALT, '/', vcfs.filtered[[i]]$ALT)[hom.alt]

    hom.ref <- which(vcfs.filtered2[[i]]$GT == '0/0')
    het.alt <- which(vcfs.filtered2[[i]]$GT == '0/1')
    hom.alt <- which(vcfs.filtered2[[i]]$GT == '1/1')
    
    vcfs.filtered2[[i]]$GENO <- 'N/N'
    vcfs.filtered2[[i]]$GENO[hom.ref] <- paste0(vcfs.filtered2[[i]]$REF, '/', vcfs.filtered2[[i]]$REF)[hom.ref]
    vcfs.filtered2[[i]]$GENO[het.alt] <- paste0(vcfs.filtered2[[i]]$REF, '/', vcfs.filtered2[[i]]$ALT)[het.alt]
    vcfs.filtered2[[i]]$GENO[hom.alt] <- paste0(vcfs.filtered2[[i]]$ALT, '/', vcfs.filtered2[[i]]$ALT)[hom.alt]


    
}

names(vcfs.filtered) <- names(vcfs)
names(vcfs.filtered2) <- names(vcfs)

In [12]:
vcfs %>% names
vcfs[[1]] %>% head
vcfs[[1]] %>% dim

[1] "D2301776" "D2303583" "K2300654" "K2300655" "K2300656" "K2300657"

,CHROM,POS,ID,REF,ALT,GT,AD,DP,AF,POS.OUT
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>
1,chr1,48186,id1,T,G,0/1,4,4,0.500,"48,186"
2,chr1,51431,id2,C,T,0/0,0,0,NA,"51,431"
3,chr1,51476,id3,T,C,0/0,0,0,NA,"51,476"
4,chr1,60796,id6,C,G,0/0,0,0,NA,"60,796"
5,chr1,63671,id7,G,A,0/1,19,19,0.421,"63,671"
6,chr1,63697,id8,T,C,0/1,19,19,0.421,"63,697"


[1] 1624518      10

In [13]:
vcfs.filtered %>% names
vcfs.filtered[[1]] %>% head
vcfs.filtered[[1]] %>% dim

[1] "D2301776" "D2303583" "K2300654" "K2300655" "K2300656" "K2300657"

,CHROM,POS,ID,REF,ALT,GT,AD,DP,AF,POS.out,GENO
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>
13,chr1,84244,id16,A,C,0/0,14,14,0.000,"84,244",A/A
20,chr1,261753,id26,G,A,0/1,46,46,0.348,"261,753",G/A
23,chr1,266225,id30,C,A,0/1,28,28,0.714,"266,225",C/A
37,chr1,665098,id50,G,A,0/0,10,10,0.000,"665,098",G/G
58,chr1,852019,id78,G,T,1/1,31,31,1.000,"852,019",T/T
59,chr1,852464,id79,C,T,1/1,18,18,1.000,"852,464",T/T


[1] 131955     11

In [14]:
# load parsed.flow

parsed.flow.raw <- read.csv(file.path(args$data.dir, 'parsed_flow.csv'))
parsed.flow <- list()

for (i in 1:23) {
    chr = paste0('chr', i)
    parsed.flow[[i]] <- parsed.flow.raw[parsed.flow.raw$V7 == chr, ][,-7]
}

names(parsed.flow) <- chrs

In [15]:
parsed.flow %>% names
parsed.flow[[1]] %>% head
parsed.flow[[1]] %>% dim

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"

,V1,V2,V3,V4,V5,V6
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A|B,A|C,B|D,A|D,B|D,A|C
2,A|B,A|C,B|D,A|D,B|D,A|C
3,A|B,A|C,B|D,A|D,B|D,A|C
4,A|B,A|C,B|D,A|D,B|D,A|C
5,A|B,A|C,B|D,A|D,B|D,A|C
6,A|B,A|C,B|D,A|D,B|D,A|C


[1] 7536    6

In [16]:
# load parsed.geno

parsed.geno.raw <- read.csv(file.path(args$data.dir, 'parsed_geno.csv'))
parsed.geno <- list()

for (i in 1:23) {
    chr = paste0('chr', i)
    parsed.geno[[i]] <- parsed.geno.raw[parsed.geno.raw$V7 == chr, ][,-7]
}

names(parsed.geno) <- chrs

In [17]:
parsed.geno %>% names
parsed.geno[[1]] %>% head
parsed.geno[[1]] %>% dim

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"

,V1,V2,V3,V4,V5,V6
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A|A,A|C,A|A,NA|NA,A|A,NA|NA
2,A|G,A|G,NA|NA,G|G,G|G,NA|NA
3,G|G,G|A,G|G,NA|NA,G|G,NA|NA
4,T|T,T|T,NA|NA,NA|NA,T|T,T|T
5,T|T,T|T,NA|NA,NA|NA,T|T,NA|NA
6,C|A,C|A,A|A,C|A,NA|NA,A|A


[1] 7536    6

In [18]:
# load is.corrected

is.corrected.raw <- read.csv(file.path(args$data.dir, 'is_corrected.csv'))
is.corrected <- list()

for (i in 1:23) {
    chr = paste0('chr', i)
    is.corrected[[i]] <- is.corrected.raw[is.corrected.raw$V13 == chr, ][,-13]
}

names(is.corrected) <- chrs

In [19]:
is.corrected %>% names
is.corrected[[1]] %>% head
is.corrected[[1]] %>% dim

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE
2,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE
3,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
4,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
5,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
6,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE


[1] 7536   12

In [21]:
# load map.list

map.list.raw <- read.csv(file.path(args$data.dir, 'map_list.csv'))
map.list <- list()

for (i in 1:23) {
    chr = paste0('chr', i)
    map.list[[i]] <- map.list.raw[map.list.raw$'chr' == chr, ][,-4]
}

names(map.list) <- chrs

In [22]:
map.list[[1]] %>% head
map.list[[1]] %>% dim

,id,pos,pos.out
,<chr>,<int>,<chr>
1,id16,84244,"84,244"
2,id26,261753,"261,753"
3,id50,665098,"665,098"
4,id78,852019,"852,019"
5,id79,852464,"852,464"
6,id89,877937,"877,937"


[1] 7536    3

# Make a plot

In [23]:
colors = brewer.pal(brewer.pal.info[args$color.palette,]$maxcolors, args$color.palette)

letters <- unique(unlist(strsplit(unique(unlist(parsed.flow)), '')))
letters <- letters[!(letters %in% c('|', 'X'))]
letter.colors <- c(colors[1:length(letters)], 'white')
names(letter.colors) <- c(letters, 'X')

fig <- get.haplo.profiles()

In [24]:
haplotype_demo <- do.subplot(fig, ncol = 1, panning = .015, margin = .007, overridehovermode = 'X unified')

# Save the plot

In [26]:
saveWidget(haplotype_demo, file.path(args$out.dir, "haplotype_demo.html"))